In [10]:
from ib_insync import *
from datetime import datetime, timedelta
from calendar import monthrange
from dateutil.relativedelta import relativedelta
import pandas as pd

In [11]:
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

<IB connected to 127.0.0.1:7497 clientId=1>

In [12]:
def loadData(contract, name, start, end, barSize='1 min'):
    df = pd.DataFrame(columns=['instrument','date','open','high','low','close','volume','average','barCount'])
    while end >= start:
        print('Got ' +end.strftime('%Y-%m-%d'))
        bars = ib.reqHistoricalData(
            contract, endDateTime=end.strftime('%Y%m%d')+' 23:59:00', 
            durationStr='1 D', barSizeSetting=barSize, whatToShow='MIDPOINT', useRTH=True)
        end -= timedelta(days=1)
        df = df.append(util.df(bars))
    df.rename(columns={'date' : 'time'}, inplace=True)
    return df.sort_values('time', axis=0).drop_duplicates(subset=['time'])

In [13]:
forex_pairs = ['USDZAR', 'USDTRY', 'USDMXN', 'USDILS', 'USDCNH', 'USDSGD EURPLN', 'EURCZK', 'EURHUF']

In [14]:
for pair in forex_pairs:
    contract = Forex(pair)
    data = loadData(contract, pair, datetime.strptime('20180101', '%Y%m%d'), datetime.strptime('20211205', '%Y%m%d'))
    data.to_csv(f'{pair}.csv', index=False)

Got 2021-12-05
Got 2021-12-04
Got 2021-12-03
Got 2021-12-02
Got 2021-12-01
Got 2021-11-30
Got 2021-11-29
Got 2021-11-28
Got 2021-11-27
Got 2021-11-26
Got 2021-11-25
Got 2021-11-24
Got 2021-11-23
Got 2021-11-22
Got 2021-11-21
Got 2021-11-20
Got 2021-11-19
Got 2021-11-18
Got 2021-11-17
Got 2021-11-16
Got 2021-11-15
Got 2021-11-14
Got 2021-11-13
Got 2021-11-12
Got 2021-11-11
Got 2021-11-10
Got 2021-11-09
Got 2021-11-08
Got 2021-11-07
Got 2021-11-06
Got 2021-11-05
Got 2021-11-04
Got 2021-11-03
Got 2021-11-02
Got 2021-11-01
Got 2021-10-31
Got 2021-10-30
Got 2021-10-29
Got 2021-10-28
Got 2021-10-27
Got 2021-10-26
Got 2021-10-25
Got 2021-10-24
Got 2021-10-23
Got 2021-10-22
Got 2021-10-21
Got 2021-10-20
Got 2021-10-19
Got 2021-10-18
Got 2021-10-17
Got 2021-10-16
Got 2021-10-15
Got 2021-10-14
Got 2021-10-13
Got 2021-10-12
Got 2021-10-11
Got 2021-10-10
Got 2021-10-09
Got 2021-10-08
Got 2021-10-07
Got 2021-10-06
Got 2021-10-05
Got 2021-10-04
Got 2021-10-03
Got 2021-10-02
Got 2021-10-01
Got 2021-0

Got 2020-06-06
Got 2020-06-05
Got 2020-06-04
Got 2020-06-03
Got 2020-06-02
Got 2020-06-01
Got 2020-05-31
Got 2020-05-30
Got 2020-05-29
Got 2020-05-28
Got 2020-05-27
Got 2020-05-26
Got 2020-05-25
Got 2020-05-24
Got 2020-05-23
Got 2020-05-22
Got 2020-05-21
Got 2020-05-20
Got 2020-05-19
Got 2020-05-18
Got 2020-05-17
Got 2020-05-16
Got 2020-05-15
Got 2020-05-14
Got 2020-05-13
Got 2020-05-12
Got 2020-05-11
Got 2020-05-10
Got 2020-05-09
Got 2020-05-08
Got 2020-05-07
Got 2020-05-06
Got 2020-05-05
Got 2020-05-04
Got 2020-05-03
Got 2020-05-02
Got 2020-05-01
Got 2020-04-30
Got 2020-04-29
Got 2020-04-28
Got 2020-04-27
Got 2020-04-26
Got 2020-04-25
Got 2020-04-24
Got 2020-04-23
Got 2020-04-22
Got 2020-04-21
Got 2020-04-20
Got 2020-04-19
Got 2020-04-18
Got 2020-04-17
Got 2020-04-16
Got 2020-04-15
Got 2020-04-14
Got 2020-04-13
Got 2020-04-12
Got 2020-04-11
Got 2020-04-10
Got 2020-04-09
Got 2020-04-08
Got 2020-04-07
Got 2020-04-06
Got 2020-04-05
Got 2020-04-04
Got 2020-04-03
Got 2020-04-02
Got 2020-0

Got 2018-12-07
Got 2018-12-06
Got 2018-12-05
Got 2018-12-04
Got 2018-12-03
Got 2018-12-02
Got 2018-12-01
Got 2018-11-30
Got 2018-11-29
Got 2018-11-28
Got 2018-11-27
Got 2018-11-26
Got 2018-11-25
Got 2018-11-24
Got 2018-11-23
Got 2018-11-22
Got 2018-11-21
Got 2018-11-20
Got 2018-11-19
Got 2018-11-18
Got 2018-11-17
Got 2018-11-16
Got 2018-11-15
Got 2018-11-14
Got 2018-11-13
Got 2018-11-12
Got 2018-11-11
Got 2018-11-10
Got 2018-11-09
Got 2018-11-08
Got 2018-11-07
Got 2018-11-06
Got 2018-11-05
Got 2018-11-04
Got 2018-11-03
Got 2018-11-02
Got 2018-11-01
Got 2018-10-31
Got 2018-10-30
Got 2018-10-29
Got 2018-10-28
Got 2018-10-27
Got 2018-10-26
Got 2018-10-25
Got 2018-10-24
Got 2018-10-23
Got 2018-10-22
Got 2018-10-21
Got 2018-10-20
Got 2018-10-19
Got 2018-10-18
Got 2018-10-17
Got 2018-10-16
Got 2018-10-15
Got 2018-10-14
Got 2018-10-13
Got 2018-10-12
Got 2018-10-11
Got 2018-10-10
Got 2018-10-09
Got 2018-10-08
Got 2018-10-07
Got 2018-10-06
Got 2018-10-05
Got 2018-10-04
Got 2018-10-03
Got 2018-1

Got 2021-05-13
Got 2021-05-12
Got 2021-05-11
Got 2021-05-10
Got 2021-05-09
Got 2021-05-08
Got 2021-05-07
Got 2021-05-06
Got 2021-05-05
Got 2021-05-04
Got 2021-05-03
Got 2021-05-02
Got 2021-05-01
Got 2021-04-30
Got 2021-04-29
Got 2021-04-28
Got 2021-04-27
Got 2021-04-26
Got 2021-04-25
Got 2021-04-24
Got 2021-04-23
Got 2021-04-22
Got 2021-04-21
Got 2021-04-20
Got 2021-04-19
Got 2021-04-18
Got 2021-04-17
Got 2021-04-16
Got 2021-04-15
Got 2021-04-14
Got 2021-04-13
Got 2021-04-12
Got 2021-04-11
Got 2021-04-10
Got 2021-04-09
Got 2021-04-08
Got 2021-04-07
Got 2021-04-06
Got 2021-04-05
Got 2021-04-04
Got 2021-04-03
Got 2021-04-02
Got 2021-04-01
Got 2021-03-31
Got 2021-03-30
Got 2021-03-29
Got 2021-03-28
Got 2021-03-27
Got 2021-03-26
Got 2021-03-25
Got 2021-03-24
Got 2021-03-23
Got 2021-03-22
Got 2021-03-21
Got 2021-03-20
Got 2021-03-19
Got 2021-03-18
Got 2021-03-17
Got 2021-03-16
Got 2021-03-15
Got 2021-03-14
Got 2021-03-13
Got 2021-03-12
Got 2021-03-11
Got 2021-03-10
Got 2021-03-09
Got 2021-0

Got 2019-11-13
Got 2019-11-12
Got 2019-11-11
Got 2019-11-10
Got 2019-11-09
Got 2019-11-08
Got 2019-11-07
Got 2019-11-06
Got 2019-11-05
Got 2019-11-04
Got 2019-11-03
Got 2019-11-02
Got 2019-11-01
Got 2019-10-31
Got 2019-10-30
Got 2019-10-29
Got 2019-10-28
Got 2019-10-27
Got 2019-10-26
Got 2019-10-25
Got 2019-10-24
Got 2019-10-23
Got 2019-10-22
Got 2019-10-21
Got 2019-10-20
Got 2019-10-19
Got 2019-10-18
Got 2019-10-17
Got 2019-10-16
Got 2019-10-15
Got 2019-10-14
Got 2019-10-13
Got 2019-10-12
Got 2019-10-11
Got 2019-10-10
Got 2019-10-09
Got 2019-10-08
Got 2019-10-07
Got 2019-10-06
Got 2019-10-05
Got 2019-10-04
Got 2019-10-03
Got 2019-10-02
Got 2019-10-01
Got 2019-09-30
Got 2019-09-29
Got 2019-09-28
Got 2019-09-27
Got 2019-09-26
Got 2019-09-25
Got 2019-09-24
Got 2019-09-23
Got 2019-09-22
Got 2019-09-21
Got 2019-09-20
Got 2019-09-19
Got 2019-09-18
Got 2019-09-17
Got 2019-09-16
Got 2019-09-15
Got 2019-09-14
Got 2019-09-13
Got 2019-09-12
Got 2019-09-11
Got 2019-09-10
Got 2019-09-09
Got 2019-0

...

Got 2020-10-19
Got 2020-10-18
Got 2020-10-17
Got 2020-10-16
Got 2020-10-15
Got 2020-10-14
Got 2020-10-13
Got 2020-10-12
Got 2020-10-11
Got 2020-10-10
Got 2020-10-09
Got 2020-10-08
Got 2020-10-07
Got 2020-10-06
Got 2020-10-05
Got 2020-10-04
Got 2020-10-03
Got 2020-10-02
Got 2020-10-01
Got 2020-09-30
Got 2020-09-29
Got 2020-09-28
Got 2020-09-27
Got 2020-09-26
Got 2020-09-25
Got 2020-09-24
Got 2020-09-23
Got 2020-09-22
Got 2020-09-21
Got 2020-09-20
Got 2020-09-19
Got 2020-09-18
Got 2020-09-17
Got 2020-09-16
Got 2020-09-15
Got 2020-09-14
Got 2020-09-13
Got 2020-09-12
Got 2020-09-11
Got 2020-09-10
Got 2020-09-09
Got 2020-09-08
Got 2020-09-07
Got 2020-09-06
Got 2020-09-05
Got 2020-09-04
Got 2020-09-03
Got 2020-09-02
Got 2020-09-01
Got 2020-08-31
Got 2020-08-30
Got 2020-08-29
Got 2020-08-28
Got 2020-08-27
Got 2020-08-26
Got 2020-08-25
Got 2020-08-24
Got 2020-08-23
Got 2020-08-22
Got 2020-08-21
Got 2020-08-20
Got 2020-08-19
Got 2020-08-18
Got 2020-08-17
Got 2020-08-16
Got 2020-08-15
Got 2020-0

Got 2019-04-21
Got 2019-04-20
Got 2019-04-19
Got 2019-04-18
Got 2019-04-17
Got 2019-04-16
Got 2019-04-15
Got 2019-04-14
Got 2019-04-13
Got 2019-04-12
Got 2019-04-11
Got 2019-04-10
Got 2019-04-09
Got 2019-04-08
Got 2019-04-07
Got 2019-04-06
Got 2019-04-05
Got 2019-04-04
Got 2019-04-03
Got 2019-04-02
Got 2019-04-01
Got 2019-03-31
Got 2019-03-30
Got 2019-03-29
Got 2019-03-28
Got 2019-03-27
Got 2019-03-26
Got 2019-03-25
Got 2019-03-24
Got 2019-03-23
Got 2019-03-22
Got 2019-03-21
Got 2019-03-20
Got 2019-03-19
Got 2019-03-18
Got 2019-03-17
Got 2019-03-16
Got 2019-03-15
Got 2019-03-14
Got 2019-03-13
Got 2019-03-12
Got 2019-03-11
Got 2019-03-10
Got 2019-03-09
Got 2019-03-08
Got 2019-03-07
Got 2019-03-06
Got 2019-03-05
Got 2019-03-04
Got 2019-03-03
Got 2019-03-02
Got 2019-03-01
Got 2019-02-28
Got 2019-02-27
Got 2019-02-26
Got 2019-02-25
Got 2019-02-24
Got 2019-02-23
Got 2019-02-22
Got 2019-02-21
Got 2019-02-20
Got 2019-02-19
Got 2019-02-18
Got 2019-02-17
Got 2019-02-16
Got 2019-02-15
Got 2019-0

Got 2021-09-25
Got 2021-09-24
Got 2021-09-23
Got 2021-09-22
Got 2021-09-21
Got 2021-09-20
Got 2021-09-19
Got 2021-09-18
Got 2021-09-17
Got 2021-09-16
Got 2021-09-15
Got 2021-09-14
Got 2021-09-13
Got 2021-09-12
Got 2021-09-11
Got 2021-09-10
Got 2021-09-09
Got 2021-09-08
Got 2021-09-07
Got 2021-09-06
Got 2021-09-05
Got 2021-09-04
Got 2021-09-03
Got 2021-09-02
Got 2021-09-01
Got 2021-08-31
Got 2021-08-30
Got 2021-08-29
Got 2021-08-28
Got 2021-08-27
Got 2021-08-26
Got 2021-08-25
Got 2021-08-24
Got 2021-08-23
Got 2021-08-22
Got 2021-08-21
Got 2021-08-20
Got 2021-08-19
Got 2021-08-18
Got 2021-08-17
Got 2021-08-16
Got 2021-08-15
Got 2021-08-14
Got 2021-08-13
Got 2021-08-12
Got 2021-08-11
Got 2021-08-10
Got 2021-08-09
Got 2021-08-08
Got 2021-08-07
Got 2021-08-06
Got 2021-08-05
Got 2021-08-04
Got 2021-08-03
Got 2021-08-02
Got 2021-08-01
Got 2021-07-31
Got 2021-07-30
Got 2021-07-29
Got 2021-07-28
Got 2021-07-27
Got 2021-07-26
Got 2021-07-25
Got 2021-07-24
Got 2021-07-23
Got 2021-07-22
Got 2021-0

Got 2020-03-27
Got 2020-03-26
Got 2020-03-25
Got 2020-03-24
Got 2020-03-23
Got 2020-03-22
Got 2020-03-21
Got 2020-03-20
Got 2020-03-19
Got 2020-03-18
Got 2020-03-17
Got 2020-03-16
Got 2020-03-15
Got 2020-03-14
Got 2020-03-13
Got 2020-03-12
Got 2020-03-11
Got 2020-03-10
Got 2020-03-09
Got 2020-03-08
Got 2020-03-07
Got 2020-03-06
Got 2020-03-05
Got 2020-03-04
Got 2020-03-03
Got 2020-03-02
Got 2020-03-01
Got 2020-02-29
Got 2020-02-28
Got 2020-02-27
Got 2020-02-26
Got 2020-02-25
Got 2020-02-24
Got 2020-02-23
Got 2020-02-22
Got 2020-02-21
Got 2020-02-20
Got 2020-02-19
Got 2020-02-18
Got 2020-02-17
Got 2020-02-16
Got 2020-02-15
Got 2020-02-14
Got 2020-02-13
Got 2020-02-12
Got 2020-02-11
Got 2020-02-10
Got 2020-02-09
Got 2020-02-08
Got 2020-02-07
Got 2020-02-06
Got 2020-02-05
Got 2020-02-04
Got 2020-02-03
Got 2020-02-02
Got 2020-02-01
Got 2020-01-31
Got 2020-01-30
Got 2020-01-29
Got 2020-01-28
Got 2020-01-27
Got 2020-01-26
Got 2020-01-25
Got 2020-01-24
Got 2020-01-23
Got 2020-01-22
Got 2020-0

Got 2018-09-27
Got 2018-09-26
Got 2018-09-25
Got 2018-09-24
Got 2018-09-23
Got 2018-09-22
Got 2018-09-21
Got 2018-09-20
Got 2018-09-19
Got 2018-09-18
Got 2018-09-17
Got 2018-09-16
Got 2018-09-15
Got 2018-09-14
Got 2018-09-13
Got 2018-09-12
Got 2018-09-11
Got 2018-09-10
Got 2018-09-09
Got 2018-09-08
Got 2018-09-07
Got 2018-09-06
Got 2018-09-05
Got 2018-09-04
Got 2018-09-03
Got 2018-09-02
Got 2018-09-01
Got 2018-08-31
Got 2018-08-30
Got 2018-08-29
Got 2018-08-28
Got 2018-08-27
Got 2018-08-26
Got 2018-08-25
Got 2018-08-24
Got 2018-08-23
Got 2018-08-22
Got 2018-08-21
Got 2018-08-20
Got 2018-08-19
Got 2018-08-18
Got 2018-08-17
Got 2018-08-16
Got 2018-08-15
Got 2018-08-14
Got 2018-08-13
Got 2018-08-12
Got 2018-08-11
Got 2018-08-10
Got 2018-08-09
Got 2018-08-08
Got 2018-08-07
Got 2018-08-06
Got 2018-08-05
Got 2018-08-04
Got 2018-08-03
Got 2018-08-02
Got 2018-08-01
Got 2018-07-31
Got 2018-07-30
Got 2018-07-29
Got 2018-07-28
Got 2018-07-27
Got 2018-07-26
Got 2018-07-25
Got 2018-07-24
Got 2018-0

In [4]:
futures = ['ES','NQ','CL','NG','HO','XB','HG','GC','SI','ZN','ZF','ED']

In [5]:
for ticker in futures:
    data = loadData(contract, ticker, datetime(2018, 11, 1), datetime(2021, 11, 15))
    df.rename(columns={'date' : 'time'})
    data.to_csv(f'{ticker}.csv', index=False)

Error 321, reqId 3: Error validating request.-'bT' : cause - Please enter exchange, contract: Future(symbol='ES', lastTradeDateOrContractMonth='CME')
Error 321, reqId 4: Error validating request.-'bT' : cause - Please enter exchange, contract: Future(symbol='ES', lastTradeDateOrContractMonth='CME')


Got 2021-11-14


AttributeError: 'NoneType' object has no attribute 'append'

In [4]:
def loadFutures(symbol, exchange, last_date, longevity, release_interval, start, end, barSize='1 min'):
    future_end = last_date
    while future_end > start:
        future_start = future_end - longevity
        end_month = future_end.strftime('%Y%m')
        contract = Future(symbol, future_end.strftime('%Y%m'), exchange)
        current_day = min(end, last_date)
        bars = ib.reqHistoricalData(
            contract, endDateTime=end.strftime('%Y%m%d %H:%m:%S'), 
            durationStr='1 D', barSizeSetting=barSize, whatToShow='TRADES', useRTH=True)
        df = util.df(bars)
        while current_day > min(start, future_start):
            current_day -= timedelta(days=1)
            bars = ib.reqHistoricalData(
                contract, endDateTime=end.strftime('%Y%m%d %H:%m:%S'), 
                durationStr='1 D', barSizeSetting=barSize, whatToShow='TRADES', useRTH=True)
            try:
                data = util.df(bars)
                df = df.append(util.df(bars))
                print('Got ' +current_day.strftime('%Y-%m-%d'))
            except:
                print('Did NOT get ' +current_day.strftime('%Y-%m-%d'))
                pass
        df.rename(columns={'date' : 'time'}, inplace=True)
        df = df.sort_values('time', axis=0).drop_duplicates(subset=['time'])
        df.to_csv(f"{symbol}-{future_end.strftime('%Y')[2:]}.{future_end.strftime('%m')}.csv", index=False)
        future_end -= release_interval

In [24]:
def loadFullFutures(symbol, exchange, last_date, longevity, release_interval, releases, barSize='1 min'):
    future_end = last_date
    while releases > 0:
        future_start = future_end - longevity
        end_month = future_end.strftime('%Y%m')
        contract = Future(symbol, future_end.strftime('%Y%m'), exchange, includeExpired=True)
        current_day = datetime(future_end.year, future_end.month, monthrange(future_end.year, future_end.month)[1])
        df = pd.DataFrame(columns=['instrument','date','open','high','low','close','volume','average','barCount'])
        while (current_day.year > future_start.year) or \
        ((current_day.year == future_start.year) and (current_day.month >= future_start.month)):
            current_day -= timedelta(days=1)
            bars = ib.reqHistoricalData(
                contract, endDateTime=current_day.strftime('%Y%m%d %H:%m:%S'), 
                durationStr='1 D', barSizeSetting=barSize, whatToShow='TRADES', useRTH=True)
            try:
                data = util.df(bars)
                df = df.append(util.df(bars))
                print('Got ' +current_day.strftime('%Y-%m-%d'))
            except:
                print('Did NOT get ' +current_day.strftime('%Y-%m-%d'))
                pass
        df.rename(columns={'date' : 'time'}, inplace=True)
        df = df.sort_values('time', axis=0).drop_duplicates(subset=['time'])
        df.to_csv(f"{symbol}-{future_end.strftime('%y')}.{future_end.strftime('%m')}.csv", index=False)
        future_end -= release_interval
        releases -= 1

In [ ]:
bars = loadFullFutures('ES', 'GLOBEX', datetime(2019, 9, 17), relativedelta(months=3), relativedelta(months=3), 10)

In [82]:
df = pd.read_csv('ES-21.12.csv', index_col='time')
# df = df.drop(columns='Unnamed: 0')
# df = df.rename(columns={'date' : 'time'})
# df = df.sort_values('time', axis=0)
# df = df.drop_duplicates(subset=['time'])
# df.to_csv('ES-21.12.csv', index=False)

In [83]:
df.head()

,open,high,low,close,volume,average,barCount
time,,,,,,,
2021-11-19 17:30:00,4700.75,4702.25,4695.00,4696.75,21634.0,4699.400,4633
2021-11-19 17:31:00,4696.75,4697.50,4693.25,4693.25,11554.0,4695.775,2984
2021-11-19 17:32:00,4693.50,4696.50,4693.25,4696.25,7553.0,4695.250,1920
2021-11-19 17:33:00,4696.00,4696.75,4692.25,4692.75,5412.0,4694.625,1401
2021-11-19 17:34:00,4693.00,4695.00,4691.50,4695.00,5903.0,4693.250,1537


In [17]:
bars = loadFullFutures('NQ', 'GLOBEX', datetime(2021, 9, 17), relativedelta(months=15), relativedelta(months=3), 1)

NameError: name 'end' is not defined

In [45]:
contract = Future('ES', '201812', 'GLOBEX', includeExpired=True)
# contract = Contract(symbol='ES', 
#                     secType='FUT', 
#                     exchange='GLOBEX', 
#                     lastTradeDateOrContractMonth='202012', 
#                     localSymbol='ESZ0',
#                     currency='USD',
#                     includeExpired=True)
bars = ib.reqHistoricalData(
            contract, endDateTime='20190517 00:00:00', 
            durationStr='1 D', barSizeSetting='1 min', whatToShow='TRADES', useRTH=True)

Error 200, reqId 8473: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='201812', exchange='GLOBEX', includeExpired=True)
Peer closed connection.


In [3]:
contract = Contract(symbol='AMD', currency='USD', secType='STK', exchange='SMART')
bars = ib.reqHistoricalData(contract, endDateTime='20190517 00:00:00', durationStr='1 D', barSizeSetting='1 min', whatToShow='TRADES', useRTH=True)

Error 162, reqId 3: Historical Market Data Service error message:No market data permissions for ISLAND STK, contract: Contract(secType='STK', symbol='AMD', exchange='SMART', currency='USD')
